In [2]:
import os
import h5py as h5
import numpy as np
import nrrd
import matplotlib.pyplot as plt

In [3]:
# file_seg = "../data/160_10-layer/volumes/00/160_10-layer_00_predictions.nrrd"
file_raw = "/Volumes/LaCie/scratch/160_10-layer/volumes/00/160_10-layer_00.h5"

assert os.path.exists(file_raw)

In [4]:
vol = {}
vol_config = {}
with h5.File(file_raw,"r") as f:
    print(f.keys())
    for key in f.keys():
        print(key)
        if key == "raw":
            vol[key] = f[key][:]
            print(f[key].shape)
            print(f[key].nbytes/1024**2)
            print(f[key].dtype)
        if key == "volume_data":
            group = f[key]
            for subkey in group.keys():
                subarray = group[subkey][...]
                vol_config[subkey] = subarray

<KeysViewHDF5 ['raw', 'volume_data']>
raw
(500, 2300, 2300)
5044.9371337890625
uint16
volume_data


In [5]:
raw = vol["raw"]
raw.shape

(500, 2300, 2300)

In [6]:


def extract_grid(volume, num_rows, num_cols):
    """
    Extract a grid of squares from a 2D NumPy array.

    Parameters:
    - image: 2D NumPy array representing the image
    - num_rows: Number of rows in the grid
    - num_cols: Number of columns in the grid

    Returns:
    - List of subarrays representing the grid squares
    """
    _, rows, cols = volume.shape
    square_size_row = rows // num_rows
    square_size_col = cols // num_cols
    subarrays = []
    for i in range(num_rows):
        row_subarrays=[]
        for j in range(num_cols):
            subarray = volume[:,i * square_size_row:(i + 1) * square_size_row,
                             j * square_size_col:(j + 1) * square_size_col]
            row_subarrays.append(subarray)
        subarrays.append(row_subarrays)
    return subarrays



In [7]:
num_rows = 10  # Example: 4 rows in the grid
num_cols = 10  # Example: 4 columns in the grid

raw_subarrays = extract_grid(raw, num_rows, num_cols)
# seg_subarrays = extract_grid(seg, num_rows, num_cols)

In [8]:
def create_nrrd_header(size, spacing, data_type='float', space='left-posterior-superior'):
    header = {
        'type': data_type,
        'dimension': len(size),
        'space dimension': len(size),
        'sizes': size,
        'space': space,
    }

    if len(spacing) == len(size):
        header['space directions'] = np.diag(spacing).tolist()
        header['kinds'] = ['domain'] * len(size)

    header.update({
        'endian': 'little',
        'encoding': 'raw',
    })

    return header


In [9]:
def numpy_dtype_to_nrrd_dtype(dtype):
    """
    Convert NumPy dtype to NRRD dtype string.

    Parameters:
    - dtype: NumPy dtype object

    Returns:
    - NRRD dtype string
    """
    dtype_mapping = {
        np.uint8: 'uint8',
        np.uint16: 'uint16',
        np.uint32: 'uint32',
        np.uint64: 'uint64',
        np.int8: 'int8',
        np.int16: 'int16',
        np.int32: 'int32',
        np.int64: 'int64',
        np.float16: 'float16',
        np.float32: 'float32',
        np.float64: 'float64',
    }

    return dtype_mapping.get(dtype, 'unknown')


In [10]:
vol_config

{'Datatype': array(b'uint16', dtype=object),
 'HistogramUpToDate': array(True),
 'Max': array(65535),
 'Min': array(0),
 'Offset': array(347.095),
 'Origin': array([0, 0, 0]),
 'Scale': array(10.9314),
 'SizeX': array(2300),
 'SizeY': array(2300),
 'SizeZ': array(500),
 'UsedBits': array(16),
 'VoxelSizeX': array(0.020618),
 'VoxelSizeY': array(0.020618),
 'VoxelSizeZ': array(0.020618)}

In [12]:
path = os.path.join(os.path.dirname(file_raw))
file_name = os.path.splitext(os.path.basename(file_raw))[0]



flag = False

VoxelSizeX = vol_config["VoxelSizeX"]
VoxelSizeY = vol_config["VoxelSizeY"]
VoxelSizeZ = vol_config["VoxelSizeZ"]

padding_row = raw.shape[1]//num_rows
padding_col = raw.shape[2]//num_cols
for n, ( raw_subs) in enumerate(raw_subarrays):
    for k, ( raw_sub) in enumerate(raw_subs):     
        row_start = n*padding_row
        row_end = (n+1)*padding_row

        col_start = k*padding_col
        col_end = (k+1)*padding_col
        raw_sub = raw_sub



        size = raw_sub.shape
        spacing = (VoxelSizeZ,VoxelSizeY,VoxelSizeX)
        data_type = numpy_dtype_to_nrrd_dtype(raw_sub.dtype)

        space = 'left-posterior-superior'

        header = create_nrrd_header(size, spacing, data_type, space)
        

        # fileout = file_name + "_%04d-%04d_%04d-%04d" %(row_start,row_end,col_start,col_end)
        # print(fileout,"%01d%01d" %(n,k))

        vol_config["SizeZ"] = raw_sub.shape[0]
        vol_config["SizeY"] = raw_sub.shape[1]
        vol_config["SizeX"] = raw_sub.shape[2]

        vol_config["Min"] = np.min(raw_sub)
        vol_config["Max"] = np.max(raw_sub)
        
        vol_config["RangeZ0"] = 0
        vol_config["RangeZ1"] = 500


        vol_config["RangeY0"] = row_start
        vol_config["RangeY1"] = row_end

        vol_config["RangeX0"] = col_start
        vol_config["RangeX1"] = col_end

        vol_config["Origin"] = (0,row_start,col_start)

        if n == 2 and k == 3:
            print(raw_sub.shape)

            fileout = file_name + "_%04d-%04d_%04d-%04d" %(row_start,row_end,col_start,col_end)
            print(fileout,"%01d%01d" %(n,k))

            nrrd.write(os.path.join(path,fileout + ".nrrd"), raw_sub,header=header)
            with h5.File(os.path.join(path,fileout + ".h5"), 'w') as fout:
                

                fout.create_dataset("raw", data = raw_sub,compression="gzip")

                volume_group = fout.create_group('volume_data')
                for key, value in vol_config.items():
                    volume_group.create_dataset(key, data=value)



(500, 230, 230)
160_10-layer_00_0460-0690_0690-0920 23


In [10]:
square_size_col = 230
square_size_row = 230

i = 5
j = 5


subarray = vol["raw"][:,i * square_size_row:(i + 1) * square_size_row,j * square_size_col:(j + 1) * square_size_col]

print(subarray.shape)
print(i * square_size_row,(i + 1) * square_size_row,j * square_size_col,(j + 1) * square_size_col)
plt.imshow(subarray[:,0,:],cmap="gray")

(2300, 0, 230)
1150 1380 1150 1380


IndexError: index 0 is out of bounds for axis 1 with size 0